In [1]:
%%capture
!pip install torch_snippets==0.462

In [2]:
%load_ext autoreload
%autoreload 2
from torch_snippets import *

if not P().resolve().stem == 'AutoTrain':
    %cd ..

/mnt/d04d289c-d0a7-4fd4-b883-ff227b3a8695/githubs/AutoTrain


In [3]:
!cat configs/classification.ini

[project]
owner: sizhky
name: horses_vs_zebras
location: ${project.name}
model_directory: ${project.location}/models
type: classification
num_classes: 2

[label_studio]
base_data_dir: /tmp/data/

[label_studio_ml]
app_name: ${project.name}/my_ml_model
script_path: auto_train_classification/script_classification.py

[online_training]
save_dir: ${project.model_directory}/model.pt

[architecture]
image_size: 224
[architecture.backbone]
model: efficientnet_b0
# everytime you change the model, vector size 
# must match the model's backbone output.
vector_size: 1280
[architecture.head]
@head: custom_head
head_input_size: ${architecture.backbone.vector_size}
num_classes: ${project.num_classes}

[training]
dir: ${project.name}/data/
train_ratio: 0.8
[training.preprocess]
@preprocess_function: my_preprocess
image_size: ${architecture.image_size}

[training.data]
train_dir: ${training.dir}/train
validation_dir: ${training.dir}/validation

[training.scheme]
epochs: 20
initial_checkpoint: ${online

In [4]:
!git clone https://github.com/rwightman/pytorch-image-models timm
%cd timm
!git checkout 54e90e8
!git reset --hard
%cd ..

fatal: destination path 'timm' already exists and is not an empty directory.
/mnt/d04d289c-d0a7-4fd4-b883-ff227b3a8695/githubs/AutoTrain/timm
HEAD is now at 54e90e8 Another attempt at sgd momentum test passing...
HEAD is now at 54e90e8 Another attempt at sgd momentum test passing...
/mnt/d04d289c-d0a7-4fd4-b883-ff227b3a8695/githubs/AutoTrain


In [5]:
os.environ['CONFIG'] = 'configs/classification.ini'
!python auto_train_classification/config_to_timm.py

2021-09-12 14:50:33.307 | DEBUG    | __main__:<module>:24 - 
{
    "model": "efficientnet_b0",
    "data_dir": "horses_vs_zebras/data/",
    "num_classes": 2,
    "epochs": 20,
    "mixup": 0.5,
    "cutmix": 0.5,
    "mixup_switch_prob": 0.30000000000000004,
    "amp": true,
    "aa": "rand-m9-mstd0.5",
    "output": "horses_vs_zebras/models/trained-model-v1",
    "initial_checkpoint": "horses_vs_zebras/models/model.pt",
    "batch_size": 2
}


In [9]:
!python timm/train.py -c auto_train_classification/config.yml horses_vs_zebras/data/train/

Training with a single process on 1 GPUs.
Loaded state_dict from checkpoint 'horses_vs_zebras/models/model.pt'
Model efficientnet_b0 created, param count:4010110
Data processing configuration for current model + dataset:
	input_size: (3, 224, 224)
	interpolation: bicubic
	mean: (0.485, 0.456, 0.406)
	std: (0.229, 0.224, 0.225)
	crop_pct: 0.875
Using native Torch AMP. Training in mixed precision.
Scheduled epochs: 20
/home/yyr/anaconda3/lib/python3.7/site-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/home/yyr/anaconda3/lib/python3.7/site-packages/torchvision/transforms/functional.py:365: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of

In [12]:
!python auto_train_classification/infer.py horses_vs_zebras/data/testing

2021-09-12 14:56:54.613 | INFO     | torch_snippets.paths:inner:24 - 1 files found at horses_vs_zebras/models/trained-model-v1
horses_vs_zebras/models/trained-model-v1/20210912-145526-efficientnet_b0-224
2021-09-12 14:56:54.684 | INFO     | __main__:predict:75 - Predicting on 16 files...
|    |   confidence |   prediction | path                                            | image_id          |
|---:|-------------:|-------------:|:------------------------------------------------|:------------------|
|  0 |     0.908771 |            0 | horses_vs_zebras/data/testing/n02381460_20.jpg  | n02381460_20.jpg  |
|  1 |     0.536447 |            0 | horses_vs_zebras/data/testing/n02381460_40.jpg  | n02381460_40.jpg  |
|  2 |     0.979523 |            0 | horses_vs_zebras/data/testing/n02381460_50.jpg  | n02381460_50.jpg  |
|  3 |     0.906846 |            1 | horses_vs_zebras/data/testing/n02391049_200.jpg | n02391049_200.jpg |
|  4 |     0.723682 |            1 | horses_vs_zebras/data/testing/n0